Librerías y Funciones propias:

In [1]:
import sys
import os
sys.path.append(os.path.abspath('..')) 

import numpy as np
import pyroomacoustics as pra
from scipy.io import wavfile
import pandas as pd
from algoritmos.CC import timeAveragedcrossCorrelation
from algoritmos.GCC import generalizedCrossCorrelation
from algoritmos.DOA import doa
from helpers.obtainSourcePositionFromAngle import obtainSourcePositionFromAngle
from helpers.addNoise import addNoise

Señales de entrada:

In [2]:

fs, sourceSignal = wavfile.read("../audios/p262_001.wav")
sourceSignal = addNoise(sourceSignal, 30)

if sourceSignal.dtype != np.float32 and sourceSignal.dtype != np.float64:
    sourceSignal = sourceSignal.astype(np.float32)

if np.max(np.abs(sourceSignal)) > 1:
    sourceSignal = sourceSignal / np.max(np.abs(sourceSignal))

Creamos los rooms:

In [3]:
heightDifferencesArraySourceDf = pd.read_excel("../datos y resultados/Datos.xlsx", engine='openpyxl', header=None)
heightDifferencesArraySource = heightDifferencesArraySourceDf.iloc[5, 4:105].to_numpy()

distancesArraySourceDf = pd.read_excel("../datos y resultados/Datos.xlsx", engine='openpyxl', header=None)
distancesArraySource = distancesArraySourceDf.iloc[6, 4:105].to_numpy()

anglesArraySourceDf = pd.read_excel("../datos y resultados/Datos.xlsx", engine='openpyxl', header=None)
anglesArraySource = anglesArraySourceDf.iloc[7, 4:105].to_numpy()

roomsHeightDifference = []

for heightDifference in heightDifferencesArraySource:

    # Room dimensions (width * length)
    roomDimensions = [8, 8, 3]

    # Create a room
    rt60Target = 0.5
    eAbsorption, max_order = pra.inverse_sabine(rt60Target, roomDimensions)
    room = pra.ShoeBox(
        roomDimensions, fs=fs, materials=pra.Material(eAbsorption), max_order=max_order
    )

    # Sound source position (x,y,z)
    sourcePosition = [6.828, 5.328, (1.5 + heightDifference)]

    # Add the source to the room
    room.add_source(sourcePosition, signal=sourceSignal)

    # Define a linear Microphone Array: 4 mics in a row
    distanceBetweenMics = 0.15
    initialX = 4 - (3 * distanceBetweenMics) / 2
    initialMicPosition = [initialX, 2.5, 1]

    # Calculate positions for 4 microphones
    micPositions = np.array([
        [initialMicPosition[0] + i * distanceBetweenMics,
        initialMicPosition[1],
        initialMicPosition[2]]
        for i in range(4)
    ])

    # Add microphones to the room
    room.add_microphone_array(pra.MicrophoneArray(micPositions.T, room.fs))

    roomsHeightDifference.append(room)

roomsDistanceArraySource = []

for distanceArraySource in distancesArraySource:

    # Room dimensions (width * length)
    roomDimensions = [8, 8, 3]

    # Create a room
    rt60Target = 0.5
    eAbsorption, max_order = pra.inverse_sabine(rt60Target, roomDimensions)
    room = pra.ShoeBox(
        roomDimensions, fs=fs, materials=pra.Material(eAbsorption), max_order=max_order
    )

    # Centro del array
    arrayCenterX = 4
    arrayCenterY = 2.5

    # Coordenadas con ángulo de 45°
    delta = distanceArraySource / np.sqrt(2)
    sourceX = arrayCenterX + delta
    sourceY = arrayCenterY + delta
    sourceZ = 1.5
    sourcePosition = [sourceX, sourceY, sourceZ]

    # Add the source to the room
    room.add_source(sourcePosition, signal=sourceSignal)

    # Define a linear Microphone Array: 4 mics in a row
    distanceBetweenMics = 0.15
    initialX = 4 - (3 * distanceBetweenMics) / 2
    initialMicPosition = [initialX, 2.5, 1]

    # Calculate positions for 4 microphones
    micPositions = np.array([
        [initialMicPosition[0] + i * distanceBetweenMics,
        initialMicPosition[1],
        initialMicPosition[2]]
        for i in range(4)
    ])

    # Add microphones to the room
    room.add_microphone_array(pra.MicrophoneArray(micPositions.T, room.fs))

    roomsDistanceArraySource.append(room)

roomsAngleArraySource = []

for angleArraySource in anglesArraySource:

    # Room dimensions (width * length)
    roomDimensions = [8, 8, 3]

    # Create a room
    rt60Target = 0.5
    eAbsorption, max_order = pra.inverse_sabine(rt60Target, roomDimensions)
    room = pra.ShoeBox(
        roomDimensions, fs=fs, materials=pra.Material(eAbsorption), max_order=max_order
    )

    # Centro del array
    arrayCenterX = 4
    arrayCenterY = 2.5

    # Coordenadas con ángulo de X grados
    sourceX, sourceY = obtainSourcePositionFromAngle(angleArraySource, 4, arrayCenterX, arrayCenterY)
    sourcePosition = [sourceX, sourceY, 3]

    # Add the source to the room
    room.add_source(sourcePosition, signal=sourceSignal)

    # Define a linear Microphone Array: 4 mics in a row
    distanceBetweenMics = 0.15
    initialX = 4 - (3 * distanceBetweenMics) / 2
    initialMicPosition = [initialX, 2.5, 1]

    # Calculate positions for 4 microphones
    micPositions = np.array([
        [initialMicPosition[0] + i * distanceBetweenMics,
        initialMicPosition[1],
        initialMicPosition[2]]
        for i in range(4)
    ])

    # Add microphones to the room
    room.add_microphone_array(pra.MicrophoneArray(micPositions.T, room.fs))

    roomsAngleArraySource.append(room)


Simulamos los rooms

In [4]:

micSignalsArraysHeightDifferece = []

for room in roomsHeightDifference:
    # Run the room simulation
    room.simulate()

    # Get the microphone signals
    micSignals = room.mic_array.signals
    micSignalsArraysHeightDifferece.append(micSignals)

micSignalsArraysDistanceArraySource = []

for room in roomsDistanceArraySource:
    # Run the room simulation
    room.simulate()

    # Get the microphone signals
    micSignals = room.mic_array.signals
    micSignalsArraysDistanceArraySource.append(micSignals)

micSignalsArraysAngleArraySource = []

for room in roomsAngleArraySource:
    # Run the room simulation
    room.simulate()

    # Get the microphone signals
    micSignals = room.mic_array.signals
    micSignalsArraysAngleArraySource.append(micSignals)

# Revisar un room simulado
room = roomsHeightDifference[0]

print("Sample rate:", room.fs)
print("Duración de la señal (s):", len(room.sources[0].signal) / room.fs)
print("Forma de micSignals:", micSignalsArraysHeightDifferece[0].shape)

for i in range(4):
    mic = micSignalsArraysHeightDifferece[0][i]
    print(f"Mic {i} - max: {np.max(np.abs(mic))}, min: {np.min(mic)}")

Sample rate: 48000
Duración de la señal (s): 2.348520833333333
Forma de micSignals: (4, 181530)
Mic 0 - max: 0.7128116594113534, min: -0.7128116594113534
Mic 1 - max: 0.8594231891203548, min: -0.6841364950668067
Mic 2 - max: 0.9925557541818442, min: -0.6880633935855668
Mic 3 - max: 0.942358127421931, min: -0.6530003692956255


Algoritmos

Time Averaged Cross Correlation

In [ ]:
delays = []
for i in range(3):
    delays.append((timeAveragedcrossCorrelation(micSignals[i], micSignals[i + 1], returnDelay=True, graphs=False)[1]) / fs)

print("Delays between microphones (in seconds):", delays)

print("DOA time averaged cross correlation: ", doa(delays, distanceBetweenMics))

GCC: Classic

In [5]:
distanceBetweenMics = [0.15, 0.30, 0.45, 0.15, 0.30, 0.15]

doaClassicHeightDifferece = []

for micSignals in micSignalsArraysHeightDifferece:
    delays = []

    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[1], mode='classic', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[2], mode='classic', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[3], mode='classic', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[2], mode='classic', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[3], mode='classic', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[2], micSignals[3], mode='classic', returnDelay=True, graphs=False)[1]) / fs)

    print("Delays between microphones (in seconds):", delays)

    print("DOA classic cross correlation: ", doa(delays, distanceBetweenMics))

    doaClassicHeightDifferece.append(doa(delays, distanceBetweenMics))

doaClassicDistanceArraySource = []

for micSignals in micSignalsArraysDistanceArraySource:
    delays = []
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[1], mode='classic', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[2], mode='classic', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[3], mode='classic', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[2], mode='classic', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[3], mode='classic', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[2], micSignals[3], mode='classic', returnDelay=True, graphs=False)[1]) / fs)

    print("Delays between microphones (in seconds):", delays)

    print("DOA classic cross correlation: ", doa(delays, distanceBetweenMics))

    doaClassicDistanceArraySource.append(doa(delays, distanceBetweenMics))

doaClassicAngleArraySource = []

for micSignals in micSignalsArraysAngleArraySource:
    delays = []
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[1], mode='classic', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[2], mode='classic', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[3], mode='classic', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[2], mode='classic', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[3], mode='classic', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[2], micSignals[3], mode='classic', returnDelay=True, graphs=False)[1]) / fs)

    print("Delays between microphones (in seconds):", delays)

    print("DOA classic cross correlation: ", doa(delays, distanceBetweenMics))

    doaClassicAngleArraySource.append(doa(delays, distanceBetweenMics))


dfHeightDifferece = pd.DataFrame(doaClassicHeightDifferece, columns=['DOA_Classic_HeightDifferece'])
dfDistanceArraySource = pd.DataFrame(doaClassicDistanceArraySource, columns=['DOA_Classic_DistanceArraySource'])
dfAngleArraySource = pd.DataFrame(doaClassicAngleArraySource, columns=['DOA_Classic_AngleArraySource'])

with pd.ExcelWriter('../datos y resultados/doa_results.xlsx', mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    dfHeightDifferece.to_excel(writer, sheet_name='DOA_Classic_HeightDifferece', index=False)
    dfDistanceArraySource.to_excel(writer, sheet_name='DOA_Classic_DistanceArraySource', index=False)
    dfAngleArraySource.to_excel(writer, sheet_name='DOA_Classic_AngleArraySource', index=False)

Delays between microphones (in seconds): [0.00014583333333333335, 6.25e-05, 0.00020833333333333335, 0.000125, 0.00014583333333333335, 0.000125]
DOA classic cross correlation:  77.41168519623177
Delays between microphones (in seconds): [0.00014583333333333335, 6.25e-05, 0.0003125, 0.000125, 0.00014583333333333335, 0.000125]
DOA classic cross correlation:  76.63785792716557
Delays between microphones (in seconds): [0.00014583333333333335, 6.25e-05, 0.0002708333333333333, 0.00014583333333333335, 0.00016666666666666666, 0.000125]
DOA classic cross correlation:  76.23925379260062
Delays between microphones (in seconds): [0.00014583333333333335, 6.25e-05, 0.0002708333333333333, 0.00014583333333333335, 0.00016666666666666666, 0.000125]
DOA classic cross correlation:  76.23925379260062
Delays between microphones (in seconds): [0.000125, 0.0, 0.0002916666666666667, 0.00010416666666666667, 0.000125, 0.00014583333333333335]
DOA classic cross correlation:  78.17833093966324
Delays between micropho

GCC: Roth

In [6]:
doaRothHeightDifferece = []

for micSignals in micSignalsArraysHeightDifferece:
    delays = []

    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[1], mode='roth', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[2], mode='roth', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[3], mode='roth', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[2], mode='roth', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[3], mode='roth', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[2], micSignals[3], mode='roth', returnDelay=True, graphs=False)[1]) / fs)

    print("Delays between microphones (in seconds):", delays)

    print("DOA Roth: ", doa(delays, distanceBetweenMics))

    doaRothHeightDifferece.append(doa(delays, distanceBetweenMics))

doaRothDistanceArraySource = []

for micSignals in micSignalsArraysDistanceArraySource:
    delays = []
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[1], mode='roth', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[2], mode='roth', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[3], mode='roth', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[2], mode='roth', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[3], mode='roth', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[2], micSignals[3], mode='roth', returnDelay=True, graphs=False)[1]) / fs)

    print("Delays between microphones (in seconds):", delays)

    print("DOA Roth: ", doa(delays, distanceBetweenMics))

    doaRothDistanceArraySource.append(doa(delays, distanceBetweenMics))

doaRothAngleArraySource = []

for micSignals in micSignalsArraysAngleArraySource:
    delays = []
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[1], mode='roth', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[2], mode='roth', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[3], mode='roth', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[2], mode='roth', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[3], mode='roth', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[2], micSignals[3], mode='roth', returnDelay=True, graphs=False)[1]) / fs)

    print("Delays between microphones (in seconds):", delays)

    print("DOA Roth: ", doa(delays, distanceBetweenMics))

    doaRothAngleArraySource.append(doa(delays, distanceBetweenMics))


dfHeightDifferece = pd.DataFrame(doaRothHeightDifferece, columns=['DOA_Roth_HeightDifferece'])
dfDistanceArraySource = pd.DataFrame(doaRothDistanceArraySource, columns=['DOA_Roth_DistanceArraySource'])
dfAngleArraySource = pd.DataFrame(doaRothAngleArraySource, columns=['DOA_Roth_AngleArraySource'])

with pd.ExcelWriter('../datos y resultados/doa_results.xlsx', mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    dfHeightDifferece.to_excel(writer, sheet_name='DOA_Roth_HeightDifferece', index=False)
    dfDistanceArraySource.to_excel(writer, sheet_name='DOA_Roth_DistanceArraySource', index=False)
    dfAngleArraySource.to_excel(writer, sheet_name='DOA_Roth_AngleArraySource', index=False)

Delays between microphones (in seconds): [0.00014583333333333335, 0.000625, -0.0005, 0.0003125, 0.0006041666666666667, 0.0016875]
Ignorado: delay=0.001687, distancia=0.150, argumento=3.859
DOA Roth:  63.60282618479867
Ignorado: delay=0.001687, distancia=0.150, argumento=3.859
Delays between microphones (in seconds): [0.00022916666666666666, 0.000625, 0.0022708333333333335, 0.0003125, -0.0008333333333333334, 0.0003125]
Ignorado: delay=0.002271, distancia=0.450, argumento=1.731
DOA Roth:  70.77838521939422
Ignorado: delay=0.002271, distancia=0.450, argumento=1.731
Delays between microphones (in seconds): [0.0003125, 0.000625, -0.0005208333333333333, 0.0003125, 0.0006041666666666667, 0.000125]
DOA Roth:  61.043954612315694
Delays between microphones (in seconds): [0.0003125, 0.000625, -0.0005416666666666666, 0.0003125, -0.0008541666666666667, 0.0003125]
DOA Roth:  76.58816924828284
Delays between microphones (in seconds): [0.0003125, 0.000625, 0.0009166666666666666, 0.0003125, 0.000541666

GCC: SCOT

In [7]:
doaScotHeightDifferece = []

for micSignals in micSignalsArraysHeightDifferece:
    delays = []

    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[1], mode='scot', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[2], mode='scot', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[3], mode='scot', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[2], mode='scot', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[3], mode='scot', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[2], micSignals[3], mode='scot', returnDelay=True, graphs=False)[1]) / fs)

    print("Delays between microphones (in seconds):", delays)

    print("DOA SCOT: ", doa(delays, distanceBetweenMics))

    doaScotHeightDifferece.append(doa(delays, distanceBetweenMics))

doaScotDistanceArraySource = []

for micSignals in micSignalsArraysDistanceArraySource:
    delays = []
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[1], mode='scot', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[2], mode='scot', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[3], mode='scot', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[2], mode='scot', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[3], mode='scot', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[2], micSignals[3], mode='scot', returnDelay=True, graphs=False)[1]) / fs)

    print("Delays between microphones (in seconds):", delays)

    print("DOA SCOT: ", doa(delays, distanceBetweenMics))

    doaScotDistanceArraySource.append(doa(delays, distanceBetweenMics))

doaScotAngleArraySource = []

for micSignals in micSignalsArraysAngleArraySource:
    delays = []
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[1], mode='scot', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[2], mode='scot', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[3], mode='scot', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[2], mode='scot', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[3], mode='scot', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[2], micSignals[3], mode='scot', returnDelay=True, graphs=False)[1]) / fs)

    print("Delays between microphones (in seconds):", delays)

    print("DOA SCOT: ", doa(delays, distanceBetweenMics))

    doaScotAngleArraySource.append(doa(delays, distanceBetweenMics))


dfHeightDifferece = pd.DataFrame(doaScotHeightDifferece, columns=['DOA_SCOT_HeightDifferece'])
dfDistanceArraySource = pd.DataFrame(doaScotDistanceArraySource, columns=['DOA_SCOT_DistanceArraySource'])
dfAngleArraySource = pd.DataFrame(doaScotAngleArraySource, columns=['DOA_SCOT_AngleArraySource'])

with pd.ExcelWriter('../datos y resultados/doa_results.xlsx', mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    dfHeightDifferece.to_excel(writer, sheet_name='DOA_SCOT_HeightDifferece', index=False)
    dfDistanceArraySource.to_excel(writer, sheet_name='DOA_SCOT_DistanceArraySource', index=False)
    dfAngleArraySource.to_excel(writer, sheet_name='DOA_SCOT_AngleArraySource', index=False)

Delays between microphones (in seconds): [0.00014583333333333335, 0.000625, -0.0005, 0.0003125, 0.0006041666666666667, 0.0016875]
Ignorado: delay=0.001687, distancia=0.150, argumento=3.859
DOA SCOT:  63.60282618479867
Ignorado: delay=0.001687, distancia=0.150, argumento=3.859
Delays between microphones (in seconds): [0.00022916666666666666, 0.000625, 0.0022708333333333335, 0.0003125, -0.0008333333333333334, 0.0003125]
Ignorado: delay=0.002271, distancia=0.450, argumento=1.731
DOA SCOT:  70.77838521939422
Ignorado: delay=0.002271, distancia=0.450, argumento=1.731
Delays between microphones (in seconds): [0.00014583333333333335, 0.000625, -0.0005208333333333333, 0.0003125, 0.0006041666666666667, 0.000125]
DOA SCOT:  65.39885476210466
Delays between microphones (in seconds): [0.0003125, 0.000625, -0.0005416666666666666, 0.0003125, -0.0008541666666666667, 0.0003125]
DOA SCOT:  76.58816924828284
Delays between microphones (in seconds): [0.0003125, 0.000625, 0.0009166666666666666, 0.0003125,

GCC: Phase

In [8]:
doaPhaseHeightDifferece = []

for micSignals in micSignalsArraysHeightDifferece:
    delays = []

    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[1], mode='phase', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[2], mode='phase', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[3], mode='phase', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[2], mode='phase', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[3], mode='phase', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[2], micSignals[3], mode='phase', returnDelay=True, graphs=False)[1]) / fs)

    print("Delays between microphones (in seconds):", delays)

    print("DOA Phase: ", doa(delays, distanceBetweenMics))

    doaPhaseHeightDifferece.append(doa(delays, distanceBetweenMics))

doaPhaseDistanceArraySource = []

for micSignals in micSignalsArraysDistanceArraySource:
    delays = []
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[1], mode='phase', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[2], mode='phase', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[3], mode='phase', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[2], mode='phase', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[3], mode='phase', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[2], micSignals[3], mode='phase', returnDelay=True, graphs=False)[1]) / fs)

    print("Delays between microphones (in seconds):", delays)

    print("DOA Phase: ", doa(delays, distanceBetweenMics))

    doaPhaseDistanceArraySource.append(doa(delays, distanceBetweenMics))

doaPhaseAngleArraySource = []

for micSignals in micSignalsArraysAngleArraySource:
    delays = []
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[1], mode='phase', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[2], mode='phase', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[0], micSignals[3], mode='phase', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[2], mode='phase', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[1], micSignals[3], mode='phase', returnDelay=True, graphs=False)[1]) / fs)
    delays.append((generalizedCrossCorrelation(micSignals[2], micSignals[3], mode='phase', returnDelay=True, graphs=False)[1]) / fs)

    print("Delays between microphones (in seconds):", delays)

    print("DOA Phase: ", doa(delays, distanceBetweenMics))

    doaPhaseAngleArraySource.append(doa(delays, distanceBetweenMics))


dfHeightDifferece = pd.DataFrame(doaPhaseHeightDifferece, columns=['DOA_Phase_HeightDifferece'])
dfDistanceArraySource = pd.DataFrame(doaPhaseDistanceArraySource, columns=['DOA_Phase_DistanceArraySource'])
dfAngleArraySource = pd.DataFrame(doaPhaseAngleArraySource, columns=['DOA_Phase_AngleArraySource'])

with pd.ExcelWriter('../datos y resultados/doa_results.xlsx', mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    dfHeightDifferece.to_excel(writer, sheet_name='DOA_Phase_HeightDifferece', index=False)
    dfDistanceArraySource.to_excel(writer, sheet_name='DOA_Phase_DistanceArraySource', index=False)
    dfAngleArraySource.to_excel(writer, sheet_name='DOA_Phase_AngleArraySource', index=False)

Delays between microphones (in seconds): [0.00014583333333333335, 0.000625, -0.0005, 0.0, 0.0006041666666666667, 0.0016875]
Ignorado: delay=0.001687, distancia=0.150, argumento=3.859
DOA Phase:  72.72463862850553
Ignorado: delay=0.001687, distancia=0.150, argumento=3.859
Delays between microphones (in seconds): [0.00014583333333333335, 0.000625, 0.0022708333333333335, 0.0003125, -0.0008333333333333334, -0.0011458333333333333]
Ignorado: delay=0.002271, distancia=0.450, argumento=1.731
Ignorado: delay=-0.001146, distancia=0.150, argumento=-2.620
DOA Phase:  80.40603594003686
Ignorado: delay=0.002271, distancia=0.450, argumento=1.731
Ignorado: delay=-0.001146, distancia=0.150, argumento=-2.620
Delays between microphones (in seconds): [0.0003125, 0.000625, 0.0022708333333333335, 0.0003125, 0.0006041666666666667, -0.0011458333333333333]
Ignorado: delay=0.002271, distancia=0.450, argumento=1.731
Ignorado: delay=-0.001146, distancia=0.150, argumento=-2.620
DOA Phase:  44.870551398997435
Ignor